# Dataset Preparation

### Importing of libraries

In [37]:
import pandas as pd
import os
import pickle

import collections
from vocab import Vocab, Vectors
from wordebd import WORDEBD

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

### Importing of data

In [39]:
# Read the CSV file
TrainData = pd.read_csv('data/dataset_original/train_sent_emo_dya.csv', encoding='shift_jis')
TestData = pd.read_csv('data/dataset_original/test_sent_emo_dya.csv', encoding='utf-8')
DevData = pd.read_csv('data/dataset_original/dev_sent_emo_dya.csv', encoding='utf-8')

# Display the first three rows
print(TrainData.shape)
print(TestData.shape)
print(DevData.shape)


(12840, 12)
(3400, 12)
(1462, 12)


### Dropping of Features

Drop Old_Dialogue_ID, Old_Utterance_ID, Season, Episode, StartTime, and EndTime

In [40]:
# Define features to drop
drop_features = list(TrainData.columns[6:]) 

In [41]:
# Drop features from X_train DataFrame
TrainData = TrainData.drop(drop_features, axis=1)
TestData = TestData.drop(drop_features, axis=1)
DevData = DevData.drop(drop_features, axis=1)

In [42]:
checkFile1 = os.path.isfile("data/dump/label_encoder.pkl")
checkFile2 = os.path.isfile("data/dump/label_decoder.pkl")

key = True

if key:
    labels = sorted(set(TrainData.Emotion))
    labelEncoder = {label: i for i, label in enumerate(labels)}
    labelDecoder = {i: label for i, label in enumerate(labels)}

    pickle.dump(labelEncoder, open('data/dump/label_encoder.pkl', 'wb'))
    pickle.dump(labelDecoder, open('data/dump/label_decoder.pkl', 'wb'))
else:
    file1 = open('data/dump/label_encoder.pkl', 'rb')
    file2 = open('data/dump/label_decoder.pkl', 'rb')
    labelEncoder = pickle.load(file1)
    labelDecoder = pickle.load(file2)
    file1.close()
    file2.close()

In [43]:
labelEncoder

{'anger': 0,
 'disgust': 1,
 'fear': 2,
 'joy': 3,
 'neutral': 4,
 'sadness': 5,
 'surprise': 6}

In [44]:
def encode_labels(encoder, l):
    return encoder[l]

In [45]:
# Apply label encoding to the "Emotion" column in y_train
TrainData["Emotion"] = TrainData["Emotion"].apply(lambda x: encode_labels(labelEncoder, x))
TestData["Emotion"] = TestData["Emotion"].apply(lambda x: encode_labels(labelEncoder, x))
DevData["Emotion"] = DevData["Emotion"].apply(lambda x: encode_labels(labelEncoder, x))

In [46]:
def find_value_ranges(lst):
    value_ranges = []
    start_index = 0

    for i in range(1, len(lst)):
        if lst[i] != lst[i - 1]:
            value_ranges.append((start_index, i - 1))
            start_index = i

    # Add the last range
    value_ranges.append((start_index, len(lst) - 1))

    return value_ranges

In [47]:
rangesTrain = find_value_ranges(TrainData["Dialogue_ID"])
print(len(rangesTrain))

rangesTest = find_value_ranges(TestData["Dialogue_ID"])
print(len(rangesTest))

rangesDev = find_value_ranges(DevData["Dialogue_ID"])
print(len(rangesDev))

2160
577
270


### Speaker Encoder

Encoding speaker on train set

In [48]:
# Check if the file exists
checkFile = os.path.isfile("data/dump/speaker_encoder_train.pkl")
encodedSpeakersTrain = []

if key:
    for range_pair in rangesTrain:
        start_idx, end_idx = range_pair
        speaker_per_dialog = TrainData['Speaker'][start_idx:end_idx + 1].copy()
        speaker_feature = sorted(set(speaker_per_dialog))
        speaker_encoder = {feature: i for i, feature in enumerate(speaker_feature)}
        speaker_decoder = {i: feature for i, feature in enumerate(speaker_feature)}

        encoded_speaker = speaker_per_dialog.replace(speaker_encoder)
        encodedSpeakersTrain.append(encoded_speaker)

    # Save encoded speaker list and ranges to a file using pickle
    file_path = 'data/dump/speaker_encoder_train.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump([encodedSpeakersTrain, rangesTrain], file)
else:
    # Load encoded speaker list and ranges from the existing pickle file
    file = open('data/dump/speaker_encoder_train.pkl', "rb")
    encodedSpeakersTrain, rangesTrain = pickle.load(file)
    file.close()

Encoding speaker on test set

In [49]:
# Check if the file exists
checkFile = os.path.isfile("data/dump/speaker_encoder_test.pkl")
encodedSpeakersTest = []

if key:
    for range_pair in rangesTest:
        start_idx, end_idx = range_pair
        speaker_per_dialog = TestData['Speaker'][start_idx:end_idx + 1].copy()
        speaker_feature = sorted(set(speaker_per_dialog))
        speaker_encoder = {feature: i for i, feature in enumerate(speaker_feature)}
        speaker_decoder = {i: feature for i, feature in enumerate(speaker_feature)}

        encoded_speaker = speaker_per_dialog.replace(speaker_encoder)
        encodedSpeakersTest.append(encoded_speaker)

    # Save encoded speaker list and ranges to a file using pickle
    file_path = 'data/dump/speaker_encoder_test.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump([encodedSpeakersTest, rangesTest], file)
else:
    # Load encoded speaker list and ranges from the existing pickle file
    file = open('data/dump/speaker_encoder_test.pkl', "rb")
    encodedSpeakersTest, rangesTest = pickle.load(file)
    file.close()

In [53]:
# Check if the file exists
checkFile = os.path.isfile("data/dump/speaker_encoder_dev.pkl")
encodedSpeakersDev = []

if checkFile:
    for range_pair in rangesDev:
        start_idx, end_idx = range_pair
        speaker_per_dialog = DevData['Speaker'][start_idx:end_idx + 1].copy()
        speaker_feature = sorted(set(speaker_per_dialog))
        speaker_encoder = {feature: i for i, feature in enumerate(speaker_feature)}
        speaker_decoder = {i: feature for i, feature in enumerate(speaker_feature)}

        encoded_speaker = speaker_per_dialog.replace(speaker_encoder)
        encodedSpeakersDev.append(encoded_speaker)

    # Save encoded speaker list and ranges to a file using pickle
    file_path = 'data/dump/speaker_encoder_dev.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump([encodedSpeakersDev, rangesDev], file)
else:
    # Load encoded speaker list and ranges from the existing pickle file
    with open('data/dump/speaker_encoder_dev.pkl', "rb") as file:
        encodedSpeakersDev, rangesDev = pickle.load(file)

Apply the encoded speakers in the Train and Test data

In [54]:
# Flatten the list of encoded speakers if it is nested
encodedSpeakersTrain_flat = [item for sublist in encodedSpeakersTrain for item in sublist]
encodedSpeakersTest_flat  = [item for sublist in encodedSpeakersTest for item in sublist]
encodedSpeakersDev_flat  = [item for sublist in encodedSpeakersDev for item in sublist]

# Replace the 'Speaker' column in TrainData with the encoded speaker data
TrainData['Speaker'] = encodedSpeakersTrain_flat
TestData['Speaker'] = encodedSpeakersTest_flat
DevData['Speaker'] = encodedSpeakersDev_flat

In [55]:
TrainData['Speaker'].unique()

array([0, 1], dtype=int64)

In [56]:
TestData['Speaker'].unique()

array([0, 1], dtype=int64)

In [57]:
DevData['Speaker'].unique()

array([1, 0], dtype=int64)

### Data Pre-Processing

In [58]:
# Download NLTK data (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Make a copy of X_train_utterances
train_utterances = TrainData['Utterance']
test_utterances = TestData['Utterance']
dev_utterances = DevData['Utterance']

# Initialize the lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [59]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Tokenize
    words = word_tokenize(text)

    # Remove punctuation and non-alphabetic characters, and stop words
    words = [word for word in words if word.isalpha() and word not in stop_words]

    # Lemmatize
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join words back into a single string
    cleaned_text = ' '.join(words)
    return cleaned_text


# Apply the clean_text function to each utterance
train_utterances = train_utterances.apply(clean_text)
test_utterances = test_utterances.apply(clean_text)
dev_utterances = dev_utterances.apply(clean_text)

In [60]:
train_utterances.head()

0    also point person company transition system
1                                 must hand full
2                                               
3                       let talk little bit duty
4                                     duty right
Name: Utterance, dtype: object

In [61]:
test_utterances.head()

0                            coffee mug number bottom
1    oh monica keep track way one missing like number
2                                                    
3                                                okay
4                                   ross say elevator
Name: Utterance, dtype: object

In [62]:
dev_utterances.head()

0                  oh god lost totally lost
1                                          
2    could go bank close account cut source
3                                    genius
4                                    genius
Name: Utterance, dtype: object

Check for empty strings

In [63]:
# Count the empty string values in X_train_utterances
empty_string_count_train = (train_utterances == '').sum()
empty_string_count_test = (test_utterances == '').sum()
empty_string_count_dev = (dev_utterances == '').sum()

print(f"Empty in Train Utterances: {empty_string_count_train}")
print(f"Empty in Test Utterances: {empty_string_count_test}")
print(f"Empty in Dev Utterances: {empty_string_count_dev}")

Empty in Train Utterances: 664
Empty in Test Utterances: 170
Empty in Dev Utterances: 89


Update 'Utterance' column and remove all empty strings

In [64]:
# Update X_train with the cleaned utterances
TrainData['Utterance'] = train_utterances
TestData['Utterance'] = test_utterances
DevData['Utterance'] = dev_utterances

# Drop rows where Utterance is an empty string. Empty string value is ''.
TrainData = TrainData[TrainData['Utterance'] != '']
TestData = TestData[TestData['Utterance'] != '']
DevData = DevData[DevData['Utterance'] != '']

# Reset the index
TrainData.reset_index(drop=True, inplace=True)
TestData.reset_index(drop=True, inplace=True)
DevData.reset_index(drop=True, inplace=True)

print(f"Train data shape: {TrainData.shape}")
print(f"Test data shape: {TestData.shape}")
print(f"Dev data shape: {DevData.shape}")

Train data shape: (12176, 6)
Test data shape: (3230, 6)
Dev data shape: (1373, 6)


<b>Take note of new shape of Train and Test Data after dropping the values

Remove rows with *n* or below number of words and create a new **Dropped Words Dataset**

In [65]:
n = 2 # This number removes rows with n or less words. (Retains rows with more than n words)

# Retain rows with more than n words
Dropwords_TrainData = TrainData[TrainData['Utterance'].apply(lambda x: len(x.split()) > n)]
Dropwords_TestData = TestData[TestData['Utterance'].apply(lambda x: len(x.split()) > n)]
Dropwords_DevData = DevData[DevData['Utterance'].apply(lambda x: len(x.split()) > n)]

Dropwords_TrainData.reset_index(drop=True, inplace=True)
Dropwords_TestData.reset_index(drop=True, inplace=True)
Dropwords_DevData.reset_index(drop=True, inplace=True)

print(f"After removing rows that is {n} words or below:")
print(f"Train shape : {Dropwords_TrainData.shape}")
print(f"Test shape: {Dropwords_TestData.shape}")
print(f"Test shape: {Dropwords_DevData.shape}")

After removing rows that is 2 words or below:
Train shape : (7354, 6)
Test shape: (1973, 6)
Test shape: (858, 6)


Get random sample from train and test data

In [66]:
# Randomly select 10 rows
random_sample = TrainData.sample(n=10)[["Utterance", "Emotion"]]

random_sample

,Utterance,Emotion
11051,oh yeah shirt dirty yep,4
3911,monica,5
8154,made tea,6
4725,get pretty one get mess,4
9044,really like lot,2
1043,somewhere along way one gon na realise done ca...,4
3793,okay night leaving museum laser floyd letting ...,3
11620,know,3
8941,next caller five hour ago must going crazy,3
7230,ok funny clown funny clown nap nap wo sleep,4


In [67]:
# Randomly select 10 rows
random_sample = TestData.sample(n=10)[["Utterance", "Emotion"]]

random_sample

,Utterance,Emotion
3108,wow fortunately,6
31,coming entire room chandler,0
2327,trying figure,5
2972,yes absolutely quality,4
257,monica still turn light bedroom,4
3210,gim,0
198,question ah egg gellers war cry neighbourhood,4
2606,fallin,0
857,okay,4
939,go,0


In [68]:
# Randomly select 10 rows
random_sample = DevData.sample(n=10)[["Utterance", "Emotion"]]

random_sample

,Utterance,Emotion
195,got another one,6
967,would fine except totally,4
1020,joey tribbiani,4
759,ross feasible,4
666,mean go airport ya wan na say good bye,5
1162,ross get,6
850,would really help guy make look good,5
1227,well nobody want ghost got one house sitting a...,4
1090,magic story use wan na sex,0
1065,well said told executor person francis needed ...,4


In [69]:
# Randomly select 10 rows
random_sample = Dropwords_TrainData.sample(n=10)[["Utterance", "Emotion"]]

random_sample

,Utterance,Emotion
7087,know really funny story happened,3
5880,say shed coat separate u get know people under...,4
6536,wanted let know changed mind gon na gon na kis...,0
406,look rachel told much easier made,0
3553,hey chandler table place close come,4
4507,phoebe maid honor,4
353,well ross seems pretty clear,4
4645,kristin kristin something funnily enough even ...,3
1137,actually one small complaint,4
6497,hey figured way tell moving,4


In [70]:
# Randomly select 10 rows
random_sample = Dropwords_TestData.sample(n=10)[["Utterance", "Emotion"]]

random_sample

,Utterance,Emotion
675,joey mom phone,4
1001,soo ah eric kind photography ya,4
36,yeah anything go except ah eye gouging fish ho...,4
1132,like end era,5
1114,said call called,5
1414,got ta make sure paolo walk first guy rachel s...,3
1264,okay finish apple juice,4
1112,well heard emma stirring came make sure could ...,4
525,bring one woman baby going sue,0
908,like taking air,6


In [71]:
# Randomly select 10 rows
random_sample = Dropwords_DevData.sample(n=10)[["Utterance", "Emotion"]]

random_sample

,Utterance,Emotion
87,wait rach one,4
94,oh god would play hide seek someone know fligh...,5
332,tell tell call okay,2
404,sure new year eve two week away wait,4
14,hey estelle listen,4
203,danielle hi everybody danielle danielle everybody,4
730,course smell like wine spilled thanks wrecking...,0
171,know harm go put name,4
513,gon na gon na go find bring back,4
216,aw hey think susan person lover,4


### Division of X and y data

In [72]:
# Create DataFrame for target labels
y_train = pd.DataFrame()
y_test = pd.DataFrame()
y_dev = pd.DataFrame()

X_train = TrainData
X_test = TestData
X_dev = DevData
    
y_train["Emotion"] = TrainData["Emotion"].copy()
y_test["Emotion"] = TestData["Emotion"].copy()
y_dev["Emotion"] = DevData["Emotion"].copy()

y_train["Dialogue_ID"] = TrainData["Dialogue_ID"].copy()
y_test["Dialogue_ID"] = TestData["Dialogue_ID"].copy()
y_dev["Dialogue_ID"] = DevData["Dialogue_ID"].copy()

In [73]:
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of y_train: {y_train.shape}')
print('--')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_test: {y_test.shape}')
print('--')
print(f'Shape of X_dev: {X_dev.shape}')
print(f'Shape of y_dev: {y_dev.shape}')

Shape of X_train: (12176, 6)
Shape of y_train: (12176, 2)
--
Shape of X_test: (3230, 6)
Shape of y_test: (3230, 2)
--
Shape of X_dev: (1373, 6)
Shape of y_dev: (1373, 2)


In [74]:
# Create DataFrame for target labels
y_dropped_train = pd.DataFrame()
y_dropped_test = pd.DataFrame()
y_dropped_dev = pd.DataFrame()

X_dropped_train = Dropwords_TrainData
X_dropped_test = Dropwords_TestData
X_dropped_dev = Dropwords_DevData

y_dropped_train["Emotion"] = Dropwords_TrainData["Emotion"].copy()
y_dropped_test["Emotion"] = Dropwords_TestData["Emotion"].copy()
y_dropped_dev["Emotion"] = Dropwords_DevData["Emotion"].copy()

y_dropped_train["Dialogue_ID"] = Dropwords_TrainData["Dialogue_ID"].copy()
y_dropped_test["Dialogue_ID"] = Dropwords_TestData["Dialogue_ID"].copy()
y_dropped_dev["Dialogue_ID"] = Dropwords_DevData["Dialogue_ID"].copy()

In [75]:
print(f'Shape of X_dropped_train: {X_dropped_train.shape}')
print(f'Shape of y_dropped_train: {y_dropped_train.shape}')
print('--')
print(f'Shape of X_dropped_test: {X_dropped_test.shape}')
print(f'Shape of y_dropped_test: {y_dropped_test.shape}')
print('--')
print(f'Shape of X_dropped_dev: {X_dropped_dev.shape}')
print(f'Shape of y_dropped_dev: {y_dropped_dev.shape}')

Shape of X_dropped_train: (7354, 6)
Shape of y_dropped_train: (7354, 2)
--
Shape of X_dropped_test: (1973, 6)
Shape of y_dropped_test: (1973, 2)
--
Shape of X_dropped_dev: (858, 6)
Shape of y_dropped_dev: (858, 2)


### Output data to new csv

##### Label Encoding Data

In [76]:
# Check if the file already exists
checkFile1 = os.path.isfile("data/dump/labels_train.pkl")
checkFile2 = os.path.isfile("data/dump/labels_test.pkl")

if key:
    pickle.dump(X_train["Emotion"], open('data/dump/labels_train.pkl', 'wb'))
    pickle.dump(X_test["Emotion"], open('data/dump/labels_test.pkl', 'wb'))

##### Training and Testing Data

In [77]:
def exportDataToCSV(df, name):
    path = f'data/DatasetPreparation/{name}.csv'
    df.to_csv(path)


In [78]:
exportDataToCSV(X_train, "X_train")
exportDataToCSV(y_train, "y_train")

exportDataToCSV(X_test, "X_test")
exportDataToCSV(y_test, "y_test")

exportDataToCSV(X_dev, "X_dev")
exportDataToCSV(y_dev, "y_dev")

In [79]:
exportDataToCSV(X_dropped_train, "dropwords_X_train")
exportDataToCSV(y_dropped_train, "dropwords_y_train")

exportDataToCSV(X_dropped_test, "dropwords_X_test")
exportDataToCSV(y_dropped_test, "dropwords_y_test")

exportDataToCSV(X_dropped_dev, "dropwords_X_dev")
exportDataToCSV(y_dropped_dev, "dropwords_y_dev")